<a href="https://colab.research.google.com/github/Cristian1308/Project-Machine-Learning/blob/main/Entrenamiento_con_3_variables_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Se importan las librerías necesarias para manejo de datos, modelado, validación y métricas de evaluación para este modelado con 3 variables

In [8]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

Se carga el dataset desde un archivo csv local

In [9]:
df = pd.read_csv("card_transdata (2).csv")

Se seleccionan 3 variables predictoras relevantes:

In [10]:
variables = ['ratio_to_median_purchase_price', 'online_order', 'distance_from_home']
X = df[variables]
y = df["fraud"]

Se divide el dataset en 80% entrenamiento y 20%

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Se crea el modelo XGBoost Classifier:

In [12]:
modelo_xgb = XGBClassifier(eval_metric='logloss', random_state=42)

Se aplica validación cruzada estratificada con 5 particiones, evaluando el modelo con la métrica ROC-AUC, útil para problemas desbalanceados como el fraude

In [13]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(modelo_xgb, X_train, y_train, cv=kfold, scoring='roc_auc')

Se imprime el resultado promedio y la variabilidad del modelo sobre los subconjuntos de validación.

In [14]:
print("----- Validación cruzada (ROC-AUC) -----")
print(f"ROC-AUC promedio: {cv_scores.mean():.4f}")
print(f"Desviación estándar: {cv_scores.std():.4f}")

----- Validación cruzada (ROC-AUC) -----
ROC-AUC promedio: 0.9722
Desviación estándar: 0.0006


Se entrena el modelo con el conjunto de entrenamiento completo.

In [15]:
modelo_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

Se hacen predicciones sobre el conjunto de prueba

In [16]:
y_pred = modelo_xgb.predict(X_test)
y_prob = modelo_xgb.predict_proba(X_test)[:, 1]

Se calculan métricas de rendimiento:

In [17]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

Se muestran los resultados del modelo

In [19]:
print("\n----- Evaluación del modelo XGBoost -----")
print(f"Accuracy     : {accuracy:.4f}")
print(f"Precision    : {precision:.4f}")
print(f"Recall       : {recall:.4f}")
print(f"F1-Score     : {f1:.4f}")
print(f"ROC-AUC      : {roc_auc:.4f}")


----- Evaluación del modelo XGBoost -----
Accuracy     : 0.9737
Precision    : 0.8113
Recall       : 0.9106
F1-Score     : 0.8581
ROC-AUC      : 0.9732
